In [66]:
import torch
from torch import tensor, nn
from loading_datas import  generate_pair_sets
import torch.nn.functional as F



In [67]:

train_pairs, train_target, train_classes, test_pairs, test_target, test_classes = generate_pair_sets(1000)
print(train_target)


tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])


In [68]:
# LeNET
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor1 = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=6, kernel_size=3, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )

        self.feature_extractor2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )

        self.feature_extractor3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=2, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x1 = self.feature_extractor1(x)
        # print(x1.shape)
        x2 = self.feature_extractor2(x1)
        # print(x2.shape)
        x3 = self.feature_extractor3(x2)
        # print(x3.shape)
        x = torch.flatten(x3, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
 

In [69]:
model = LeNet5(2)
output = model(train_pairs)
print(output[1])

tensor([[0.4786, 0.5214],
        [0.4627, 0.5373],
        [0.4805, 0.5195],
        ...,
        [0.4755, 0.5245],
        [0.4681, 0.5319],
        [0.4860, 0.5140]], grad_fn=<SoftmaxBackward0>)


In [70]:
lr, nb_epochs, batch_size = 1e-1, 1000, 1000
model = LeNet5(2)
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()
for e in range(nb_epochs):
    for input, targets in zip(train_pairs.split(batch_size), train_target.split(batch_size)):
        output = model(input)[1]
        # print(output)
        loss = criterion(output, targets)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        
    optimizer.step()

tensor([[0.5067, 0.4933],
        [0.5087, 0.4913],
        [0.5160, 0.4840],
        [0.5271, 0.4729],
        [0.5287, 0.4713],
        [0.4952, 0.5048],
        [0.5034, 0.4966],
        [0.5079, 0.4921],
        [0.5101, 0.4899],
        [0.5171, 0.4829],
        [0.5123, 0.4877],
        [0.5147, 0.4853],
        [0.5144, 0.4856],
        [0.5018, 0.4982],
        [0.5354, 0.4646],
        [0.5190, 0.4810],
        [0.4914, 0.5086],
        [0.5136, 0.4864],
        [0.5137, 0.4863],
        [0.4948, 0.5052],
        [0.4941, 0.5059],
        [0.4989, 0.5011],
        [0.4983, 0.5017],
        [0.4996, 0.5004],
        [0.5218, 0.4782],
        [0.5202, 0.4798],
        [0.5210, 0.4790],
        [0.5132, 0.4868],
        [0.5155, 0.4845],
        [0.4927, 0.5073],
        [0.5005, 0.4995],
        [0.5100, 0.4900],
        [0.5181, 0.4819],
        [0.5047, 0.4953],
        [0.5072, 0.4928],
        [0.5287, 0.4713],
        [0.5134, 0.4866],
        [0.5120, 0.4880],
        [0.5

In [71]:
output = model(test_pairs)[1]
# print(output)
print(torch.round(output))
print(test_target)

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]], grad_fn=<RoundBackward0>)
tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]])


In [72]:
error = 0

for i in range(1000):
    if torch.argmax(output[i]) != torch.argmax(test_target[i]):
        error += 1

print(error)

478
